In [1]:
import pandas as pd
import numpy as np
import joblib
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
USD_RATE = 86.14  # 1 USD = 86.14 ₹

def load_models_and_data():
    """Charge les modèles et données nécessaires"""
    df = pd.read_csv('ndtv_data_final.csv')
    model = joblib.load('models/phone_price_model.pkl')
    brand_encoder = joblib.load('models/brand_encoder.pkl')
    processor_encoder = joblib.load('models/processor_encoder.pkl')
    scaler = joblib.load('models/scaler.pkl')
    feature_names = joblib.load('models/feature_names.pkl')
    return df, model, brand_encoder, processor_encoder, scaler, feature_names

def predict_phone_price(input_params, df, model, brand_encoder, processor_encoder, scaler, feature_names):
    """Fonction de prédiction principale (identique au script Streamlit)"""
    try:
        # Calcul des features dérivées
        ram_mb = input_params['ram'] * 1000
        camera_total = input_params['rear_camera'] + input_params['front_camera']
        
        price_per_gb = df['Price'].mean() / (df['Internal storage (GB)'].mean() or 1)
        price_per_mp = df['Price'].mean() / ((df['Rear camera'].mean() + df['Front camera'].mean()) or 1)
        screen_to_battery_ratio = input_params['screen_size'] / (input_params['battery'] / 1000) if input_params['battery'] else 1.0
        price_per_ram = df['Price'].mean() / (df['RAM (MB)'].mean() or 1)
        battery_to_screen_ratio = input_params['battery'] / input_params['screen_size'] if input_params['screen_size'] else 1.0
        is_premium = int(input_params['brand'] in ['Apple', 'Samsung', 'OnePlus'])

        # Préparation des données
        input_data = pd.DataFrame({
            'Brand': [input_params['brand']],
            'Battery capacity (mAh)': [input_params['battery']],
            'Screen size (inches)': [input_params['screen_size']],
            'Processor': [input_params['processor']],
            'RAM (MB)': [ram_mb],
            'Internal storage (GB)': [input_params['storage']],
            'Rear camera': [input_params['rear_camera']],
            'Front camera': [input_params['front_camera']],
            'Price_per_GB': [price_per_gb],
            'Price_per_MP': [price_per_mp],
            'Screen_to_Battery_Ratio': [screen_to_battery_ratio],
            'Camera_Total': [camera_total],
            'RAM_GB': [input_params['ram']],
            'Price_per_RAM': [price_per_ram],
            'Battery_to_Screen_Ratio': [battery_to_screen_ratio],
            'Is_Premium': [is_premium]
        })

        # Encodage et mise à l'échelle
        input_data['Brand'] = brand_encoder.transform(input_data['Brand'])
        input_data['Processor'] = [input_params['processor']]  # Conserve la valeur numérique
        input_data = input_data[feature_names]
        input_scaled = scaler.transform(input_data)

        # Prédiction
        predicted_log_price = model.predict(input_scaled)[0]
        predicted_price = np.expm1(predicted_log_price)
        predicted_usd = predicted_price / USD_RATE

        # Calcul du prix moyen similaire
        similar_phones = df[
            (df['Internal storage (GB)'].between(input_params['storage'] * 0.8, input_params['storage'] * 1.2)) &
            (df['RAM (MB)'].between(ram_mb * 0.8, ram_mb * 1.2))
        ]
        avg_price = similar_phones['Price'].mean() if not similar_phones.empty else predicted_price
        avg_usd = avg_price / USD_RATE

        return {
            'predicted_usd': predicted_usd,
            'avg_usd': avg_usd,
            'similar_phones_count': len(similar_phones),
            'input_data': input_params
        }
        
    except Exception as e:
        print(f"Erreur lors de la prédiction : {e}")
        return None

In [6]:
# Charger les données et modèles une fois
df, model, brand_encoder, processor_encoder, scaler, feature_names = load_models_and_data()

# Définir des paramètres de test (simule l'input utilisateur)
test_params = {
    'brand': 'Samsung',
    'processor': 4,
    'battery': 4500,
    'screen_size': 6.5,
    'ram': 8,
    'storage': 128,
    'rear_camera': 64,
    'front_camera': 20
}

# Faire la prédiction
results = predict_phone_price(test_params, df, model, brand_encoder, processor_encoder, scaler, feature_names)

# Afficher les résultats
if results:
    print(f"Prix prédit: ${results['predicted_usd']:.2f}")
    print(f"Prix moyen similaire: ${results['avg_usd']:.2f}")
    print(f"Nombre de téléphones similaires: {results['similar_phones_count']}")
    
    # Variation en pourcentage
    variation = ((results['predicted_usd'] - results['avg_usd']) / results['avg_usd']) * 100
    print(f"Variation: {variation:.1f}%")

Prix prédit: $415.55
Prix moyen similaire: $431.59
Nombre de téléphones similaires: 22
Variation: -3.7%


c:\Users\EAHR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(


In [8]:
# Graphique en barres
fig_bar = px.bar(
    x=['Prix Prédit', 'Prix Moyen Similaire'],
    y=[results['predicted_usd'], results['avg_usd']],
    title="Comparaison des Prix",
    labels={'x': 'Type de Prix', 'y': 'Prix (USD)'},
    color=['Prix Prédit', 'Prix Moyen Similaire'],
    color_discrete_map={'Prix Prédit': 'blue', 'Prix Moyen Similaire': 'orange'}
)
fig_bar.update_layout(template="plotly_white", showlegend=False)
fig_bar.show()

# Radar chart
fig_radar = go.Figure()
fig_radar.add_trace(go.Scatterpolar(
    r=[
        test_params['battery']/5000,
        test_params['screen_size']/7,
        test_params['ram']/8,
        test_params['storage']/256,
        test_params['rear_camera']/64,
        test_params['front_camera']/32
    ],
    theta=['Batterie', 'Écran', 'RAM', 'Stockage', 'Cam. Arrière', 'Cam. Avant'],
    fill='toself', name='Téléphone Test', line_color='#1f77b4'))
fig_radar.add_trace(go.Scatterpolar(
    r=[
        df['Battery capacity (mAh)'].mean() / 5000,
        df['Screen size (inches)'].mean() / 7,
        df['RAM (MB)'].mean() / 8000,
        df['Internal storage (GB)'].mean() / 256,
        df['Rear camera'].mean() / 64,
        df['Front camera'].mean() / 32
    ],
    theta=['Batterie', 'Écran', 'RAM', 'Stockage', 'Cam. Arrière', 'Cam. Avant'],
    fill='toself', name='Moyenne Générale', line_color='#ff7f0e'))
fig_radar.update_layout(
    template="plotly_white",
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    title="Comparaison des Caractéristiques"
)
fig_radar.show()